# 0.0. Imports

In [23]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from IPython.core.display import HTML

## 0.1. Helper Functions

In [24]:
def jupyter_settings():
    %matplotlib inline
    %pylab inline
    
    plt.style.use( 'ggplot')
    plt.rcParams['figure.figsize'] = [24, 9]
    plt.rcParams['font.size'] = 24
    
    display( HTML( '<style>.container { width:100% !important; }</style>') )
    pd.options.display.max_columns = None
    pd.options.display.max_rows = None
    pd.set_option( 'display.expand_frame_repr', False )
    
    sns.set()
    
jupyter_settings()

def describe_num(num_attributes):
    a = num_attributes.describe().T.reset_index()
    
    d4 = pd.DataFrame(num_attributes.apply( lambda x: x.max() - x.min())).T
    d5 = pd.DataFrame(num_attributes.apply( lambda x: x.skew())).T      
    d6 = pd.DataFrame(num_attributes.apply( lambda x: x.kurtosis())).T
    d7 = pd.DataFrame(num_attributes.apply( lambda x: x.unique().shape[0])).T
    
    n = pd.concat([d4, d5, d6, d7], axis=0).T.reset_index()
    n.columns = ['attibutes', 'range', 'skew', 'kurtosis', 'unique']
    b = a.merge(n, right_on='attibutes', left_on='index')
    b.drop('index', axis=1, inplace=True)
    columns = ['attibutes', 'count', 'mean', 'std', '25%', '50%', '75%', 'min', 'max', 'range',
           'skew', 'kurtosis', 'unique']
    return b[columns]

Populating the interactive namespace from numpy and matplotlib


## 0.2. Load dataset

In [30]:
df_addresses = pd.read_csv('../data/raw/tb_addresses.csv', delimiter=';')
df_banners = pd.read_csv('../data/raw/tb_banners.csv', delimiter=';')
df_cities = pd.read_csv('../data/raw/tb_cities.csv', delimiter=';')
df_coupons = pd.read_csv('../data/raw/tb_coupons.csv', delimiter=';')
df_customer_payments = pd.read_csv('../data/raw/tb_customer_payments.csv', delimiter=';')
df_ordered_products = pd.read_csv('../data/raw/tb_ordered_products.csv', delimiter=';')
df_product_tags = pd.read_csv('../data/raw/tb_product_tags.csv', delimiter=';')
df_products = pd.read_csv('../data/raw/tb_products.csv', delimiter=';')
df_ratings = pd.read_csv('../data/raw/tb_ratings.csv', delimiter=';')
df_solds_cognatis = pd.read_csv('../data/raw/tb_solds_cognatis.csv', delimiter=';')
df_users = pd.read_csv('../data/raw/tb_users.csv', delimiter=';')
df_vendor_payments = pd.read_csv('../data/raw/tb_vendor_payments.csv', delimiter=';')
df_vendors = pd.read_csv('../data/raw/tb_vendors.csv', delimiter=';')
df_vendors_delivery_fees = pd.read_csv('../data/raw/tb_vendors_delivery_fees.csv', delimiter=';')

### DF VENDORS T = DF VENDORS + DF VENDORS DELIVERY FEES + DF ADRESSES

In [31]:
df_vendors.shape

(403, 15)

In [32]:
# Juntando a tabela DF VENDORS com a VENDORS DELIVERY FEES
df_vendors_t = pd.merge(df_vendors, df_vendors_delivery_fees, on='id_fornecedor', how='left')
df_t = df_addresses.drop('id_fornecedor', axis=1)
# Juntando a tabela DF VENDORS T com a ADRESSES
df_vendors_t = pd.merge(df_vendors_t, df_t, on='id_endereco', how='left')
df_vendors_t = df_vendors_t.drop('id_endereco', axis=1)
df_vendors_t.isna().sum()

id_fornecedor               0
nome_fornecedor             0
data_registro               0
qtde_produtos               0
preco_medio                 0
avaliacao_media             0
e_aprovado                  0
nome_empresa                0
delivery_raio_km            0
delivery_sistema          561
hora_abertura               0
hora_fechamento             0
data_aprovacao            260
data_reprovacao           370
valor_fixo                175
valor_minimo              172
valor_por_km              170
valor_cobertura_minima    173
distancia_maxima          165
tipo_taxa                 165
bairro                      6
ponto_referencia          245
cidade                      0
estado                      0
uf                          0
pais                        0
tipo                        0
cep                        33
dtype: int64

In [33]:
dic = {'id_fornecedor' : 'id_fornecedor', 
        'nome_fornecedor': 'nome_fornecedor',
        'data_registro': 'forc_data_registro',
        'qtde_produtos': 'forc_qtde_produtos',
        'preco_medio' : 'forc_preco_medio',
        'avaliacao_media': 'forc_avaliacao_media',
        'e_aprovado': 'forc_e_aprovado',
        'nome_empresa': 'forc_nome_empresa',
        'delivery_raio_km': 'forc_delivery_raio_km',
        'delivery_sistema': 'forc_delivery_sistema',
        'hora_abertura': 'forc_hora_abertura',
        'hora_fechamento': 'forc_hora_fechamento', 
        'data_aprovacao': 'forc_data_aprovacao', 
        'data_reprovacao': 'forc_data_reprovacao', 
        'valor_fixo': 'forc_delivery_fees_valor_fixo',
        'valor_minimo': 'forc_delivery_fees_valor_minimo', 
        'valor_por_km': 'forc_delivery_fees_valor_por_km', 
        'valor_cobertura_minima': 'forc_delivery_fees_valor_cobertura_minima',
        'distancia_maxima': 'forc_delivery_fees_distancia_maxima', 
        'tipo_taxa': 'forc_delivery_fees_tipo_taxa', 
        'bairro': 'forc_adress_bairro', 
        'ponto_referencia': 'forc_adress_ponto_referencia', 
        'cidade': 'forc_adress_cidade',
        'estado': 'forc_adress_estado', 
        'uf': 'forc_adress_uf', 
        'pais': 'forc_adress_pais', 
        'tipo': 'forc_adress_tipo_endereco', 
        'cep': 'forc_adress_cep'}
df_vendors_t = df_vendors_t.rename(columns=dic)
df_vendors_t.shape

(626, 28)

In [34]:
df_vendors_t.sample(40)

,id_fornecedor,nome_fornecedor,forc_data_registro,forc_qtde_produtos,forc_preco_medio,forc_avaliacao_media,forc_e_aprovado,forc_nome_empresa,forc_delivery_raio_km,forc_delivery_sistema,forc_hora_abertura,forc_hora_fechamento,forc_data_aprovacao,forc_data_reprovacao,forc_delivery_fees_valor_fixo,forc_delivery_fees_valor_minimo,forc_delivery_fees_valor_por_km,forc_delivery_fees_valor_cobertura_minima,forc_delivery_fees_distancia_maxima,forc_delivery_fees_tipo_taxa,forc_adress_bairro,forc_adress_ponto_referencia,forc_adress_cidade,forc_adress_estado,forc_adress_uf,forc_adress_pais,forc_adress_tipo_endereco,forc_adress_cep
504,60f5e20d76d90d00217f1fd2,Quitutesdathah,2021-07-19 13:40:38,0,0,0,True,THRIS RIBUIRO DR SILVR 40180037811,15,NaN,12:00,20:00,2021-10-15 20:15:17,NaN,20.00,0.0,0.00,0.0,15.0,FIXED,Tucuruvi,NaN,São Paulo,São Paulo,SP,Brasil,OFFICE,1000001.0
238,5ff32e4338e1c3002114e76d,Flôrida Garden Café,2021-01-03 17:39:23,0,0,0,True,ssydrs sytuyds chmdrcwh dm gsstrhyhmws,30,NaN,10:00,18:00,2021-10-14 19:17:50,NaN,25.00,0.0,0.00,0.0,20.0,FIXED,Parque Frondoso,posto de gasolina loja embaixo do EDA,Cotia,São Paulo,SP,Brasil,OFFICE,NaN
355,607b7324cf718c0021ab19a2,Dushi by Angela Miranda,2021-04-17 18:20:35,0,0,0,False,Âygdls Fwdl Cruz Mwrsyds Dhcsrws,15,NaN,09:00,17:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Moema,NaN,São Paulo,São Paulo,SP,Brasil,OFFICE,1000001.0
159,5f90ade25418a50021045d18,Maege Confeitaria,2020-10-20 23:35:50,0,0,0,True,Rygdls Rpsrdcwds ds Cruz Swlvs,7,NaN,10:00,19:00,2020-12-08 18:13:02,2020-11-24 13:14:29,0.00,0.0,0.00,0.0,7.0,FREE,Ipiranga,"travessa da rua bom pastor, esquina da rua sil...",São Paulo,São Paulo,SP,Brasil,OFFICE,1000001.0
282,603ae312162b7a0021fab066,Café Com Bolo Doceria,2021-02-27 23:41:01,0,0,0,False,Csfé Chm Bhlh Dhcdrws,3,NaN,09:00,19:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Parque São Jorge,Próximo ao Clube Corinthians,São Paulo,São Paulo,SP,Brasil,OFFICE,1000001.0
602,617084522dd96e0022520500,NubiaCampos.Cakes1,2021-10-20 20:00:56,0,0,0,False,Rsqudl Csmphs dd Mhurs,10,NaN,10:00,17:30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Vila Primavera,NaN,São Paulo,São Paulo,SP,Brasil,OFFICE,1000001.0
225,5fe7bbe42e676f002139281e,Li Orlandin Confeitaria,2020-12-26 00:45:20,0,0,0,False,UO Prhduçõds culwyárwss,10,NaN,09:30,18:30,2021-04-07 19:45:17,2021-07-21 19:49:28,20.00,0.0,0.00,0.0,10.0,FIXED,Chácara Santo Antônio (Zona Leste),trav. rua Atucuri,São Paulo,São Paulo,SP,Brasil,OFFICE,1000001.0
489,60f3370596be590021794c25,MAMI Comfort Sweets,2021-07-17 19:29:11,0,0,0,True,MRMI DULÍCIRS,20,THIRD_PARTY,09:00,18:00,2021-07-20 13:32:42,2021-07-20 13:23:16,0.00,5.0,2.00,8.0,20.0,MIN_PLUS_PER_KM,Vila Morse,próximo ao estádio do Morumbi,São Paulo,São Paulo,SP,Brasil,OFFICE,1000001.0
617,617eeb44fed36c0022c29781,Chocolateria do Brasil,2021-10-30 22:17:56,0,0,0,False,Chhchlstdrws dh Brsswl,36,NaN,09:00,18:30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Jardim Virginia Bianca,Próx. Av Nova Cantareira alt 5000,São Paulo,São Paulo,SP,Brasil,OFFICE,1000001.0
378,608c3c52e8cdce0021ad428b,Confeitaria da Luana,2021-04-30 04:21:41,0,0,0,True,Chyfdwtsrws ds Lusys Ltds,10,NaN,10:00,22:00,2021-05-05 17:01:52,NaN,8.99,0.0,0.00,0.0,3.0,FIXED,Pinheiros,Casa de esquina,São Paulo,São Paulo,SP,Brasil,OFFICE,1000001.0


### DF PRODUCTS T = DF PRODUCTS + DF PRODUCTS TAG + DF VENDORS T

In [5]:
# Juntando a tabela DF PRODUCTS com a PRODUCTS TAG
df_product_tags = df_product_tags.rename(columns={'tag_produtos_id' : 'id_tipo'})
df_products_t = pd.merge(df_products, df_product_tags, on='id_tipo', how='left')
df_products_t = df_products_t.drop('id_tipo', axis=1)
df_products_t.isna().sum()

id_produto                     0
id_fornecedor                  0
id_disconto                 2888
nome                           0
ativo                          7
aprovado                       0
arquivado                      1
entregavel                     0
recuperavel                    0
tempo_preparacao_minutos       6
avaliacao_media               61
destacado                      1
texto_display                  1
tipo                           1
dtype: int64

In [6]:
dic = {'id_produto' : 'id_produto', 
       'id_fornecedor' : 'id_fornecedor', 
       'id_disconto' : 'id_desconto', 
       'nome' : 'prod_nome', 
       'ativo' : 'prod_ativo',
       'aprovado' : 'prod_aprovado', 
       'arquivado' : 'prod_arquivado', 
       'entregavel' : 'prod_entregavel', 
       'recuperavel' : 'prod_recuperavel',
       'tempo_preparacao_minutos' : 'prod_tempo_prep_minuto', 
       'avaliacao_media' : 'prod_avaliacao_media', 
       'destacado' : 'prod_tag_destacado',
       'texto_display' : 'prod_tag_texto_display', 
       'tipo' : 'prod_tag_tipo'}
df_products_t = df_products_t.rename(columns=dic)
df_products_t.columns

Index(['id_produto', 'id_fornecedor', 'id_desconto', 'prod_nome', 'prod_ativo',
       'prod_aprovado', 'prod_arquivado', 'prod_entregavel',
       'prod_recuperavel', 'prod_tempo_prep_minuto', 'prod_avaliacao_media',
       'prod_tag_destacado', 'prod_tag_texto_display', 'prod_tag_tipo'],
      dtype='object')

In [7]:
# Juntando a tabela DF PRODUCTS T com a DF VENDORS T
df_products_t = pd.merge(df_products_t, df_vendors_t, on='id_fornecedor', how='left')
df_products_t = df_products_t.drop('id_fornecedor', axis=1)
df_products_t.isna().sum()

id_produto                                      0
id_desconto                                  6172
prod_nome                                       0
prod_ativo                                     15
prod_aprovado                                   0
prod_arquivado                                  2
prod_entregavel                                 0
prod_recuperavel                                0
prod_tempo_prep_minuto                         13
prod_avaliacao_media                          102
prod_tag_destacado                              2
prod_tag_texto_display                          2
prod_tag_tipo                                   2
nome_fornecedor                                 0
forc_data_registro                              0
forc_qtde_produtos                              0
forc_preco_medio                                0
forc_avaliacao_media                            0
forc_e_aprovado                                 0
forc_nome_empresa                               0


### DF USERS

In [8]:
# Sugestão: ainda não fazer merge com a tabela de fornecedores para não poluir a quantidade de colunas
id_fornecedor = df_users[~df_users['id_fornecedor'].isna()]['id_fornecedor'].tolist()
print('Usuários que são também vendedores: ', df_vendors[df_vendors['id_fornecedor'].isin(id_fornecedor)].shape[0])

Usuários que são também vendedores:  403


In [9]:
dic = {'id_usuario': 'id_usuario', 
       'id_fornecedor': 'user_id_fornecedor', 
       'nome_completo': 'user_nome_completo', 
       'data_ultima_compra': 'user_data_ultima_compra',
       'qtde_total_compra': 'user_qtde_total_compra', 
       'gasto_total': 'user_gasto_total',
       'ultima_qtde_compra': 'user_ultima_qtde_compra',
       'ticket_medio': 'user_ticket_medio',
       'aceita_thrid_party_mkt': 'user_aceita_thrid_party_mkt',
       'aceita_mkt': 'user_aceita_mkt'}
df_users = df_users.rename(columns=dic)
df_users.columns

Index(['id_usuario', 'user_id_fornecedor', 'user_nome_completo',
       'user_data_ultima_compra', 'user_qtde_total_compra', 'user_gasto_total',
       'user_ultima_qtde_compra', 'user_ticket_medio',
       'user_aceita_thrid_party_mkt', 'user_aceita_mkt'],
      dtype='object')

### DF CUSTOMER PAYMENTS T = DF CUSTOMER PAYMENTS + DF USERS

In [10]:
dic = {'id_ordens' : 'id_ordem', 
       'id_pagamentos_cliente' : 'id_pagamentos_cliente', 
       'id_cliente' : 'id_usuario', 
       'data_pagamento' : 'user_pay__data_pagamento',
       'valor' : 'user_pay_valor', 
       'status_pagamento': 'user_pay_status_pagamento', 
       'metodo_pagamento': 'user_pay_metodo_pagamento'}
# Juntando DF CUSTOMER PAYMENTS com a DF USERS
df_customer_payments = df_customer_payments.rename(columns=dic)
df_customer_payments_t = pd.merge(df_customer_payments, df_users, on='id_usuario', how='left')
df_customer_payments_t = df_customer_payments_t.drop('id_usuario', axis=1)
# Não faz sentido manter se o usuário é também fornecedor para esta análise (poderá ser revisada posteriormente)
df_customer_payments_t = df_customer_payments_t.drop('user_id_fornecedor', axis=1)
df_customer_payments_t.isna().sum()

id_ordem                          0
id_pagamentos_cliente             0
user_pay__data_pagamento          0
user_pay_valor                    0
user_pay_status_pagamento         0
user_pay_metodo_pagamento         0
user_nome_completo                0
user_data_ultima_compra        1936
user_qtde_total_compra            1
user_gasto_total                  1
user_ultima_qtde_compra           1
user_ticket_medio                 1
user_aceita_thrid_party_mkt       1
user_aceita_mkt                   1
dtype: int64

### <font color='red'>DF ORDERED PRODUCTS T =  DF ORDERED PRODUCTS + DF PRODUCTS T </font>

**Tabela com os dados vendidos pela plataforma onde está vinculado:**
 - a ordem do pedido com as respectivas informações
 - o nome do produto e os respectivos dados de cadastro
 - o nome do fornecedor e os respectivos dados de cadastro
 - dados do cliente que realizou a compra

In [26]:
# JUNTANDO DF ORDERED PRODUCTS com DF PRODUCS T
dic = {'id_produto_vendido': 'id_produto_vendido', 
       'id_produto': 'id_produto', 
       'id_variacao_selecionada' : 'id_variacao_selecionada',
       'valor_medida': 'order_prod_valor_medida', 
       'tipo_medida' : 'order_prod_tipo_medida', 
       'numero_pessoas_servidas' : 'order_prod_numero_pessoas_servidas', 
       'preco' : 'order_prod_preco',
       'qtde' : 'order_prod_qtde', 
       'anotacao' : 'order_prod_anotacao', 
       'subtotal' : 'order_prod_subtotal', 
       'id_ordem' : 'id_ordem'}
df_ordered_products = df_ordered_products.rename(columns=dic)
df_ordered_products_t = pd.merge(df_ordered_products, df_products_t, on='id_produto', how='left')
df_ordered_products_t = df_ordered_products_t.drop('id_produto', axis=1)
df_ordered_products_t.isna().sum()

id_produto_vendido                               0
id_variacao_selecionada                        146
order_prod_valor_medida                        104
order_prod_tipo_medida                         104
order_prod_numero_pessoas_servidas             106
order_prod_preco                               148
order_prod_qtde                                148
order_prod_anotacao                          32317
order_prod_subtotal                            234
id_ordem                                       234
id_desconto                                  26295
prod_nome                                      162
prod_ativo                                     197
prod_aprovado                                  162
prod_arquivado                                 162
prod_entregavel                                162
prod_recuperavel                               162
prod_tempo_prep_minuto                         197
prod_avaliacao_media                           201
prod_tag_destacado             

In [27]:
df_ordered_products_t = pd.merge(df_ordered_products_t, df_customer_payments_t, on='id_ordem', how='left')
df_ordered_products_t = df_ordered_products_t.drop('id_ordem', axis=1)
df_ordered_products_t.isna().sum()

id_produto_vendido                               0
id_variacao_selecionada                        146
order_prod_valor_medida                        104
order_prod_tipo_medida                         104
order_prod_numero_pessoas_servidas             106
order_prod_preco                               148
order_prod_qtde                                148
order_prod_anotacao                          37229
order_prod_subtotal                            234
id_desconto                                  30307
prod_nome                                      162
prod_ativo                                     203
prod_aprovado                                  162
prod_arquivado                                 162
prod_entregavel                                162
prod_recuperavel                               162
prod_tempo_prep_minuto                         203
prod_avaliacao_media                           207
prod_tag_destacado                             168
prod_tag_texto_display         

In [28]:
df_ordered_products_t.shape

(39189, 61)

# <font color='red'> 1.0. Descrição dos dados </font>